<center><h1><b>5주차 ✅ 폐기물 데이터 전처리</b></h1></center>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

----
#### 발생현황

In [508]:
# 폐기물 발생현황 데이터 가져오기 (19,20년도)
# 19년: 단위(일), 20년: 단위(년)
ocr_19 = pd.read_csv('data/waste/19년_폐기물_발생현황_생활폐기물_20220610191058.csv', )
ocr_20 = pd.read_csv('data/waste/20년_폐기물_발생현황_생활폐기물_20220610190114.csv', )
ocr_19.head(2)

,시도(1),항목별(1),항목별(2),항목별(3),2019
0,합계,총계,소계,소계,45912.1
1,합계,종량제방식에의한혼합배출,소계,소계,20971.1


In [466]:
# 소계가 하나밖에 없는 행들을 찾으려고 한다.
cty_raw_19 = ocr_19[ocr_19['시도(1)'] == '합계']
cty_raw_19 =  cty_raw_19[['항목별(2)', '항목별(3)', '2019']]
cty_19 = (cty_raw_19[['항목별(2)']].value_counts() == 1)
cty_19

항목별(2)    
가연성           False
불연성           False
폐합성수지류        False
소계            False
폐지류           False
영농폐기물         False
폐섬유류           True
폐전지류           True
폐전기전자제품        True
폐의류            True
폐유리병류          True
폐식용유           True
폐가구류           True
폐고무류           True
건설폐재류          True
재활용잔재물         True
기타(배출불명 등)     True
기타             True
금속캔            True
고철류            True
폐형광등           True
dtype: bool

In [467]:
# 소계가 하나밖에 없는 제외하고 cols에 넣는다.
# 해당 컬럼은 따라서 항목별(3)에서 소계를 삭제해도 된다.
cty_cols_19 = [k[0] for k, v in cty_19.items() if not v]
cty_cols_19

['가연성', '불연성', '폐합성수지류', '소계', '폐지류', '영농폐기물']

In [ ]:
# ocr_19[~( 
#         (ocr_19['시도(1)'].str.contains('계')) | \
#         (((ocr_19['항목별(1)'] != '음식물류폐기물분리배출')\
#                 & ocr_19['항목별(2)'].isin(cty_cols_19)) \
#                 & (ocr_19['항목별(3)'].str.contains('계')))
#         )].head(50)

In [469]:
cty_raw_20 = ocr_20[ocr_20['구분(1)'] == '전국']
cty_raw_20 =  cty_raw_20[['구분(3)', '구분(4)', '2020']]
cty_20 = (cty_raw_20[['구분(3)']].value_counts() == 1)
cty_cols_20 = [k[0] for k, v in cty_20.items() if not v]
cty_cols_20


['가연성', '불연성', '폐합성수지류', '소계', '폐지류', '영농폐기물']

In [470]:
# 합계, 총계, 소계 삭제
# 항목별(3)의 소계는 삭제하면 안된다.
# 왜냐하면 카테고리가 하나뿐인 거여서 삭제하면 그 데이터 자체가 날아가는 것.
drops = (ocr_19['시도(1)'].str.contains('계')) | \
        (((ocr_19['항목별(1)'] != '음식물류폐기물분리배출')\
                & ocr_19['항목별(2)'].isin(cty_cols_19)) \
                & (ocr_19['항목별(3)'].str.contains('계')))
ocr_19 = ocr_19[~drops]
ocr_19.head()

,시도(1),항목별(1),항목별(2),항목별(3),2019
45,서울,종량제방식에의한혼합배출,가연성,음식물류폐기물,127.9
46,서울,종량제방식에의한혼합배출,가연성,폐지류,802.9
47,서울,종량제방식에의한혼합배출,가연성,폐합성수지류,530.0
48,서울,종량제방식에의한혼합배출,가연성,폐목재류,163.1
49,서울,종량제방식에의한혼합배출,가연성,폐섬유류,112.7


In [472]:
drops = (ocr_20['구분(1)'].str.contains('계')) | \
        (((ocr_20['구분(2)'] != '음식물류 폐기물 분리배출')\
                & ocr_20['구분(3)'].isin(cty_cols_20)) \
                & (ocr_20['구분(4)'].str.contains('계')))
ocr_20 = ocr_20[~drops]
ocr_20.head()

,구분(1),구분(2),구분(3),구분(4),2020
3,전국,종량제방식 등 혼합배출,가연성,폐지류,1829611.5
4,전국,종량제방식 등 혼합배출,가연성,폐합성수지류,1706740.2
5,전국,종량제방식 등 혼합배출,가연성,폐고무류,145010.4
6,전국,종량제방식 등 혼합배출,가연성,폐섬유류,370664.1
7,전국,종량제방식 등 혼합배출,가연성,음식물류 폐기물,481839.1


In [475]:
# 안쓰는 컬럼 삭제 & 컬럼명 변경하기
cols = ['시도(1)','2019']
ocr_19 = ocr_19[cols]
ocr_19 = ocr_19.rename(columns={'시도(1)':'시도명',})
ocr_19.head()

,시도명,2019
45,서울,127.9
46,서울,802.9
47,서울,530.0
48,서울,163.1
49,서울,112.7


In [476]:
cols = ['구분(1)','2020']
ocr_20 = ocr_20[cols]
ocr_20 = ocr_20.rename(columns={'구분(1)':'시도명',})
ocr_20.head()

,시도명,2020
3,전국,1829611.5
4,전국,1706740.2
5,전국,145010.4
6,전국,370664.1
7,전국,481839.1


In [477]:
# 타입 확인
ocr_19.dtypes, ocr_20.dtypes

(시도명      object
 2019    float64
 dtype: object,
 시도명      object
 2020    float64
 dtype: object)

In [478]:
# 폐기물 수 단위 맞추기 (19:일, 20:년)
ocr_19['2019'] = ocr_19['2019'] * 365
ocr_19.head(2)

,시도명,2019
45,서울,46683.5
46,서울,293058.5


In [479]:
# 시도명 기준으로 그룹핑
ocr_19_g = ocr_19.groupby('시도명').sum()
ocr_19_g

,2019
시도명,
강원,681199.5
경기,3483523.5
경남,1296772.0
경북,863991.5
광주,423473.0
대구,937283.5
대전,469645.5
부산,1077115.0
서울,3160170.0


In [480]:
ocr_20_g = ocr_20.groupby('시도명').sum()
ocr_20_g

,2020
시도명,
강원,667683.5
경기,3901516.4
경남,1270196.9
경북,903107.9
광주,420659.4
대구,938049.8
대전,498151.8
부산,1148394.2
서울,3177620.0


In [481]:
# 병합하기 전 shape 확인
ocr_19_g.shape, ocr_20_g.shape

((17, 1), (18, 1))

In [482]:
# 19,20년도 폐기물 발생현황 데이터 병합하기
ocr = pd.merge(ocr_19_g, ocr_20_g, left_index=True, right_index=True, how='outer')
ocr

,2019,2020
시도명,,
강원,681199.5,667683.5
경기,3483523.5,3901516.4
경남,1296772.0,1270196.9
경북,863991.5,903107.9
광주,423473.0,420659.4
대구,937283.5,938049.8
대전,469645.5,498151.8
부산,1077115.0,1148394.2
서울,3160170.0,3177620.0


In [483]:
# 전국, 합계 행 삭제하기
ocr = ocr.dropna()
ocr

,2019,2020
시도명,,
강원,681199.5,667683.5
경기,3483523.5,3901516.4
경남,1296772.0,1270196.9
경북,863991.5,903107.9
광주,423473.0,420659.4
대구,937283.5,938049.8
대전,469645.5,498151.8
부산,1077115.0,1148394.2
서울,3160170.0,3177620.0


In [484]:
# 폐기물 수 int 타입으로 변경
ocr = ocr[['2019', '2020']].astype('int')
ocr

,2019,2020
시도명,,
강원,681199,667683
경기,3483523,3901516
경남,1296772,1270196
경북,863991,903107
광주,423473,420659
대구,937283,938049
대전,469645,498151
부산,1077115,1148394
서울,3160170,3177620


In [485]:
# 시도명 길게 매핑하기
sidos = {
    '강원': '강원도',
    '경기': '경기도',
    '경남': '경상남도',
    '경북': '경상북도',
    '광주': '광주광역시',
    '대구': '대구광역시',
    '대전': '대전광역시',
    '부산': '부산광역시',
    '서울': '서울특별시',
    '세종': '세종특별자치시',
    '울산': '울산광역시',
    '인천': '인천광역시',
    '전남': '전라남도',
    '전북': '전라북도',
    '제주': '제주특별자치도',
    '충남': '충청남도',
    '충북': '충청북도'
}

ocr.index = ocr.index.map(sidos)
ocr.head()


,2019,2020
시도명,,
강원도,681199,667683
경기도,3483523,3901516
경상남도,1296772,1270196
경상북도,863991,903107
광주광역시,423473,420659


In [486]:
# 19,20년도 폐기물 발생현황 데이터 저장하기
ocr.to_csv('output/waste_ocr_1920.csv')

----
#### 처리현황

In [519]:
# 폐기물 처리현황 데이터 가져오기 (19,20년도)
# 19년: 단위(일), 20년: 단위(년)
prs_19 = pd.read_csv('data/waste/19년_폐기물_처리주체별_처리현황_생활폐기물_20220610200200.csv', encoding='cp949')
prs_20 = pd.read_csv('data/waste/20년_폐기물_처리주체별_처리현황_생활폐기물_20220610191442.csv', encoding='cp949')
prs_19.head(2)

,시도(1),항목별(1),항목별(2),2019
0,합계,총계,계,45912.1
1,합계,총계,매립,6753.5


In [524]:
prs_20.shape


(360, 4)

In [489]:
prs_20.head(2)


,구분(1),항목별(1),항목별(2),2020
0,합계,총계,소계,17303386.4
1,합계,총계,재활용,9762502.9


In [490]:
# 컬럼명 변경하기
prs_19 = prs_19.rename(columns={
    '시도(1)':'시도명', '항목별(1)':'처리업체', '항목별(2)':'처리방식', })
prs_19.head(2)

,시도명,처리업체,처리방식,2019
0,합계,총계,계,45912.1
1,합계,총계,매립,6753.5


In [491]:
prs_20 = prs_20.rename(columns={
    '구분(1)':'시도명', '항목별(1)':'처리업체', '항목별(2)':'처리방식', })
prs_20.head(2)


,시도명,처리업체,처리방식,2020
0,합계,총계,소계,17303386.4
1,합계,총계,재활용,9762502.9


In [492]:
# 합계, 총계, 소계 삭제하기
drops = prs_19['시도명'].str.contains('합계')\
        | prs_19['처리업체'].str.contains('총계')\
        | prs_19['처리방식'].str.contains('계')
prs_19 = prs_19[~drops]
prs_19


,시도명,처리업체,처리방식,2019
26,서울,지방자치단체,매립,830.6
27,서울,지방자치단체,소각,1892.9
28,서울,지방자치단체,재활용,1050.2
29,서울,지방자치단체,기타,0.0
31,서울,위탁처리업체,매립,139.8
...,...,...,...,...
354,제주,위탁처리업체,기타,0.0
356,제주,자가처리업체,매립,0.0
357,제주,자가처리업체,소각,0.0
358,제주,자가처리업체,재활용,4.8


In [494]:
prs_19.groupby('시도명').sum() * 365

,2019
시도명,
강원,681199.5
경기,3483523.5
경남,1296772.0
경북,863991.5
광주,423473.0
대구,937283.5
대전,469645.5
부산,1077115.0
서울,3160170.0


In [495]:
drops = prs_20['시도명'].str.contains('계')\
        | prs_20['처리업체'].str.contains('계')\
        | prs_20['처리방식'].str.contains('계')
prs_20 = prs_20[~drops]
prs_20

,시도명,처리업체,처리방식,2020
26,서울특별시,공공처리,재활용,379742.3
27,서울특별시,공공처리,소각,730391.5
28,서울특별시,공공처리,매립,344031.3
29,서울특별시,공공처리,기타,0.0
31,서울특별시,자가처리,재활용,5584.0
...,...,...,...,...
354,제주특별자치도,자가처리,기타,0.0
356,제주특별자치도,위탁처리,재활용,164243.1
357,제주특별자치도,위탁처리,소각,0.0
358,제주특별자치도,위탁처리,매립,0.0


In [496]:
# 처리업체명 고유값 알아보기 & 값 통일하기
prs_19['처리업체'].unique().tolist(), prs_20['처리업체'].unique().tolist()

(['지방자치단체', '위탁처리업체', '자가처리업체'], ['공공처리', '자가처리', '위탁처리'])

In [497]:
prs_company = {
    '지방자치단체': '공공처리',
    '자가처리업체':'자가처리',
    '위탁처리업체':'위탁처리'
}
prs_19['처리업체'] = prs_19.loc[:, '처리업체'].map(prs_company)
prs_19.head(2)

,시도명,처리업체,처리방식,2019
26,서울,공공처리,매립,830.6
27,서울,공공처리,소각,1892.9


In [498]:
# 처리방식 고유값 알아보기
prs_19['처리방식'].unique().tolist(), prs_20['처리방식'].unique().tolist()

(['매립', '소각', '재활용', '기타'], ['재활용', '소각', '매립', '기타'])

In [499]:
# 19년도 데이터 시도명 맞추기
prs_19['시도명'] = prs_19['시도명'].map(sidos)
prs_19.head(2)


,시도명,처리업체,처리방식,2019
26,서울특별시,공공처리,매립,830.6
27,서울특별시,공공처리,소각,1892.9


In [500]:
# 19,20년 처리현황 데이터 결측치 확인
prs_19.isnull().sum(), prs_20.isnull().sum()

(시도명     0
 처리업체    0
 처리방식    0
 2019    0
 dtype: int64,
 시도명     0
 처리업체    0
 처리방식    0
 2020    0
 dtype: int64)

In [501]:
# shape 확인
prs_19.shape, prs_20.shape

((204, 4), (204, 4))

In [502]:
# 19,20년도 폐기물 처리현황 데이터 병합
prs = pd.merge(prs_19, prs_20, how='outer')
prs

,시도명,처리업체,처리방식,2019,2020
0,서울특별시,공공처리,매립,830.6,344031.3
1,서울특별시,공공처리,소각,1892.9,730391.5
2,서울특별시,공공처리,재활용,1050.2,379742.3
3,서울특별시,공공처리,기타,0.0,0.0
4,서울특별시,위탁처리,매립,139.8,1522.0
...,...,...,...,...,...
199,제주특별자치도,위탁처리,기타,0.0,0.0
200,제주특별자치도,자가처리,매립,0.0,0.0
201,제주특별자치도,자가처리,소각,0.0,0.0
202,제주특별자치도,자가처리,재활용,4.8,1960.2


In [503]:
# 폐기물 처리현황 단위 맞추기
prs['2019'] = prs.loc[:, '2019'] * 365
prs

,시도명,처리업체,처리방식,2019,2020
0,서울특별시,공공처리,매립,303169.0,344031.3
1,서울특별시,공공처리,소각,690908.5,730391.5
2,서울특별시,공공처리,재활용,383323.0,379742.3
3,서울특별시,공공처리,기타,0.0,0.0
4,서울특별시,위탁처리,매립,51027.0,1522.0
...,...,...,...,...,...
199,제주특별자치도,위탁처리,기타,0.0,0.0
200,제주특별자치도,자가처리,매립,0.0,0.0
201,제주특별자치도,자가처리,소각,0.0,0.0
202,제주특별자치도,자가처리,재활용,1752.0,1960.2


In [ ]:
# 폐기물 처리현황 정수값으로 변경
# prs[['2019', '2020']] = prs.loc[:, ['2019', '2020']].astype('int')
# prs.head()

In [ ]:
# downcast
# prs.info()
# ocols = prs.select_dtypes('object').columns
# icols = prs.select_dtypes('int').columns
# prs[ocols] = prs[ocols].astype('category')
# prs[icols] = prs[icols].apply(pd.to_numeric, downcast='integer')
# prs.info()


In [505]:
# 19,20년도 폐기물 처리현황 데이터 저장하기
prs.to_csv('output/waste_prs_1920.csv', index=False)

----
#### 공공처리시설(지방자치단체) 매립/소각시설

In [533]:
# 지방자치단체 폐기물 매립/소각시설 데이터 불러오기 (19,20년도)
land_raw = pd.read_csv('data/waste/1920년_지방자치단체_폐기물_처리시설_현황_매립시설_20220610212242.csv', encoding='cp949')
fire_raw = pd.read_csv('data/waste/1920년_지방자치단체_폐기물_처리시설_현황_소각시설_20220610205703.csv', encoding='cp949')
land_raw.head(2)

,구분(1),시점,매립지수 (개소),기매립량(전년도 매립량 포함) (m³),매립량 (톤),설치비,설치비.1,연간유지관리비 (백만원),매립지 관리인원 (명)
0,구분(1),시점,소계,소계,소계,국비,지방비,소계,소계
1,부산광역시,2019,1,15566090,232256,88663,275139,4914,31


In [534]:
land_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   구분(1)                  33 non-null     object
 1   시점                     33 non-null     object
 2   매립지수 (개소)              33 non-null     object
 3   기매립량(전년도 매립량 포함) (m³)  33 non-null     object
 4   매립량 (톤)                33 non-null     object
 5   설치비                    33 non-null     object
 6   설치비.1                  33 non-null     object
 7   연간유지관리비 (백만원)          33 non-null     object
 8   매립지 관리인원 (명)           33 non-null     object
dtypes: object(9)
memory usage: 2.4+ KB


In [535]:
land_raw.shape


(33, 9)

In [338]:
# 컬럼명 변경하기 & 안쓰는 컬럼 삭제하기
fire_raw = fire_raw.rename(columns={
    '구분(1)': '시도명',
    '시설수 (개소)': '소각시설수',
    '처리량 (톤)':'처리량(톤)',
    '설치비 (백만원).1': '국비(백만원)',
    '설치비 (백만원).2': '지방비(백만원)',
    '연간유지관리비 (백만원)': '유지관리비(백만원/년)',
    '관리인원 (명)':'관리인원',
    '에너지이용실태 (Gcal)':'에너지이용(Gcal)'
})
del fire_raw['설치비 (백만원)']
fire_raw = fire_raw.drop(0) # 첫번째 행 삭제 (소계)
fire_raw.head(2)

,시도명,시점,소각시설수,처리량(톤),국비(백만원),지방비(백만원),유지관리비(백만원/년),관리인원,에너지이용(Gcal)
1,서울특별시,2019,5,713757,61302,314935,66693,315,1535848
2,서울특별시,2020,5,716241,61302,314935,79500,315,1560095


In [342]:
land_raw = land_raw.rename(columns={
    '구분(1)': '시도명',
    '매립지수 (개소)': '매립시설수',
    '기매립량(전년도 매립량 포함) (m³)': '기매립량(m³)',
    '매립량 (톤)': '매립량(톤)',
    '설치비': '국비(백만원)',
    '설치비.1': '지방비(백만원)',
    '연간유지관리비 (백만원)': '유지관리비(백만원/년)',
    '매립지 관리인원 (명)': '관리인원',
})
land_raw = land_raw.drop(0)  # 첫번째 행 삭제 (소계)
land_raw.head(2)


,시도명,시점,매립시설수,기매립량(m³),매립량(톤),국비(백만원),지방비(백만원),유지관리비(백만원/년),관리인원
1,부산광역시,2019,1,15566090,232256,88663,275139,4914,31
2,부산광역시,2020,1,15872690,229270,88663,275139,4544,33


In [344]:
fire_raw.to_csv('output/fire_1920.csv', index=False)
land_raw.to_csv('output/land_1920.csv', index=False)

----
#### 폐기물 관리구역 정보

In [72]:
# 관리구역 정보 데이터 가져오기 (19,20년도)
info_raw = pd.read_csv('data/waste/1920년_생활폐기물_관리구역_현황_20220610190948.csv', encoding='cp949')
info_raw.head()

,구분(1),항목별(1),항목별(2),2019,2020
0,합계,전체 행정구역,면적 (km²),100448.26,100458.33
1,합계,전체 행정구역,인구 (명),53090684.00,52942262.00
2,합계,전체 행정구역,동(읍·면)수 (개),3491.00,3501.00
3,합계,전체 행정구역,세대수 (가구),22481466.00,23093108.00
4,합계,생활폐기물 관리구역,면적 (km²),99649.48,99905.51


In [73]:
# 컬럼명 변경하기 & 안쓰는 컬럼 삭제하기
info_raw = info_raw.rename(columns={
    '구분(1)': '시도명',
    '항목별(1)': '구역',
    '항목별(2)': '항목',
})
info_raw.head()

,시도명,구역,항목,2019,2020
0,합계,전체 행정구역,면적 (km²),100448.26,100458.33
1,합계,전체 행정구역,인구 (명),53090684.00,52942262.00
2,합계,전체 행정구역,동(읍·면)수 (개),3491.00,3501.00
3,합계,전체 행정구역,세대수 (가구),22481466.00,23093108.00
4,합계,생활폐기물 관리구역,면적 (km²),99649.48,99905.51


In [74]:
# 합계 삭제하기
info = info_raw[~(info_raw['시도명'].str.contains('계'))]
info.head()

,시도명,구역,항목,2019,2020
16,서울특별시,전체 행정구역,면적 (km²),605.24,605.24
17,서울특별시,전체 행정구역,인구 (명),9993743.00,9911088.00
18,서울특별시,전체 행정구역,동(읍·면)수 (개),424.00,425.00
19,서울특별시,전체 행정구역,세대수 (가구),4327605.00,4417954.00
20,서울특별시,생활폐기물 관리구역,면적 (km²),605.24,605.24


In [75]:
# 구역 고유값 확인하기 & 필요한 부분만
info['구역'].unique()

array(['전체 행정구역', '생활폐기물 관리구역', '생활폐기물 관리제외지역', '관리제외지역 비율'], dtype=object)

In [76]:
info = info[~(info['구역'] == '관리제외지역 비율')]
info.head()

,시도명,구역,항목,2019,2020
16,서울특별시,전체 행정구역,면적 (km²),605.24,605.24
17,서울특별시,전체 행정구역,인구 (명),9993743.00,9911088.00
18,서울특별시,전체 행정구역,동(읍·면)수 (개),424.00,425.00
19,서울특별시,전체 행정구역,세대수 (가구),4327605.00,4417954.00
20,서울특별시,생활폐기물 관리구역,면적 (km²),605.24,605.24


In [77]:
# value 값 정수로 변경
info[['2019', '2020']] = info[['2019', '2020']].astype('int')
info.head() 

,시도명,구역,항목,2019,2020
16,서울특별시,전체 행정구역,면적 (km²),605,605
17,서울특별시,전체 행정구역,인구 (명),9993743,9911088
18,서울특별시,전체 행정구역,동(읍·면)수 (개),424,425
19,서울특별시,전체 행정구역,세대수 (가구),4327605,4417954
20,서울특별시,생활폐기물 관리구역,면적 (km²),605,605


In [78]:
# 폐기물 관리구역 정보 데이터 저장하기
info.to_csv('output/info_1920.csv', index=False)

----
#### 폐기물 관리예산

In [104]:
# 폐기물 관리예산 정보 데이터 가져오기
# 19,20년: 단위(천원)
budget_raw = pd.read_csv('data/waste/1920년_생활폐기물_관리예산_20220610191853.csv', encoding='cp949')
budget_raw.head()

,구분(1),항목별(1),항목별(2),항목별(3),2019,2020
0,합계,총계,소계,소계,6942985584,7371709122
1,합계,국비,소계,소계,96459259,109477897
2,합계,지방비,소계,소계,5474561747,5774954879
3,합계,지방비,광역시도,자체 예산,434920855,445559192
4,합계,지방비,지자체 시군구,시도지원예산,147811475,59437255


In [105]:
# 컬럼명 변경 & 쓸모없는 컬럼 삭제하기
budget_raw = budget_raw.rename(columns={
    '구분(1)': '시도명',
    '항목별(1)': '분류',
})
budget_raw = budget_raw.drop(columns=['항목별(2)', '항목별(3)'], axis=1)
budget_raw.head()

,시도명,분류,2019,2020
0,합계,총계,6942985584,7371709122
1,합계,국비,96459259,109477897
2,합계,지방비,5474561747,5774954879
3,합계,지방비,434920855,445559192
4,합계,지방비,147811475,59437255


In [106]:
# 분류 고유값 확인하기
budget_raw['분류'].unique()

array(['총계', '국비', '지방비', '치관', '지방채', '세입수수료', '기타'], dtype=object)

In [107]:
# 합계 삭제하기
drops = budget_raw['시도명'].str.contains('계') | \
            budget_raw['분류'].str.contains('계|치관|지방채|기타')
budget = budget_raw[~drops]
budget

,시도명,분류,2019,2020
11,서울특별시,국비,4771900,9883500
12,서울특별시,지방비,995989659,1009849334
13,서울특별시,지방비,145190040,141115289
14,서울특별시,지방비,6781664,4848145
15,서울특별시,지방비,844017955,863885900
...,...,...,...,...
172,제주특별자치도,지방비,144927289,175060454
173,제주특별자치도,지방비,14488750,34677976
174,제주특별자치도,지방비,2945000,0
175,제주특별자치도,지방비,127493539,140382478


In [108]:
# 시도명, 분류 기준으로 그룹화하기
budget_g = budget.groupby(by=['시도명', '분류']).sum()
budget_g.head()

2019       2020
시도명 분류                         
강원도 국비      12122744   23911725
    세입수수료   46171000   47846000
    지방비    517782926  582499714
경기도 국비       3537843    3101540
    세입수수료  308633000  366070000

In [109]:
# 폐기물 예산 데이터 저장하기
budget_g.to_csv('output/budget_1920.csv', )